## Scan PerkinElmer XML files to match image urls to compounds

In [1]:
import pathlib
import numpy as np
import pandas as pd

from utils.parse_xml import PerkinElmerXML
from utils.channels import get_channel_map

In [2]:
np.random.seed(42)

In [3]:
# Load files with platemap metadata
platemap_dir = pathlib.Path("..", "..", "1.Data-exploration", "results")

cp_platemap_file = pathlib.Path(platemap_dir, "CellPainting_compound_metadata_plate_location_with_reproducibility.tsv.gz")
l1000_platemap_file = pathlib.Path(platemap_dir, "L1000_compound_metadata_plate_location_with_reproducibility.tsv.gz")

cp_platemap_df = pd.read_csv(cp_platemap_file, sep="\t")
l1000_platemap_df = pd.read_csv(l1000_platemap_file, sep="\t")

print(cp_platemap_df.shape)
cp_platemap_df.head(3)

(36641, 20)


,compound,no_of_compounds,well,dose_recode,median_score,p_value,assay,normalization,category,pass_thresh,neg_log_10_p_val,dose,Metadata_Assay_Plate_Barcode,Metadata_Well,Metadata_Plate_Map_Name,replicate_name,Metadata_dose_recode,Metadata_broad_sample,pert_iname,moa
0,kpt-330,5,E14,5,0.92348,0.0,Cell Painting,spherized,all_data,True,3.5,3.33 uM,SQ00015157,E14,C-7161-01-LM6-004,replicate_27690,5,BRD-K21361524-001-01-1,kpt-330,exportin antagonist
1,kpt-330,5,E14,5,0.92348,0.0,Cell Painting,spherized,all_data,True,3.5,3.33 uM,SQ00015159,E14,C-7161-01-LM6-004,replicate_27696,5,BRD-K21361524-001-01-1,kpt-330,exportin antagonist
2,kpt-330,5,E14,5,0.92348,0.0,Cell Painting,spherized,all_data,True,3.5,3.33 uM,SQ00015158,E14,C-7161-01-LM6-004,replicate_27702,5,BRD-K21361524-001-01-1,kpt-330,exportin antagonist


### Select compounds of interest

We will use the following as a guide to select which Cell Painting images to visualize.

Note that we will only select one FOV per image.

- Top most reproducible compound in Cell Painting
    - And all replicates
- Top most reproducible compound in L1000
    - And all replicates
- Least reproducible compound in Cell Painting (with at least 5 replicates)
    - And all replicates
- Least reproducible compound in L1000 (with at least 3 replicates)
    - And all replicates
- Random DMSO
    - 5 random replicates across plates
    
In total, we will identify the URLs of 5 replicates * 5 channels * 5 categories = 125 total images

In [4]:
# Top most reproducible compound in Cell Painting
top_cp_compound = cp_platemap_df.compound.unique()[0]
top_cp_compound_dose = cp_platemap_df.dose[0]
print(top_cp_compound, top_cp_compound_dose)

kpt-330 3.33 uM


In [5]:
# Top most reproducible compound in L1000
top_l1000_compound = l1000_platemap_df.compound.unique()[0]
top_l1000_compound_dose = l1000_platemap_df.dose[0]
print(top_l1000_compound, top_l1000_compound_dose)

resminostat 10 uM


In [6]:
# Least reproducible compound in Cell Painting (with at least 5 replicates)
bottom_cp_compound = cp_platemap_df.query("no_of_compounds >= 5").sort_values(by="median_score").compound.unique()[0]
bottom_cp_compound_dose = cp_platemap_df.query("no_of_compounds >= 5").sort_values(by="median_score").reset_index().dose[0]
print(bottom_cp_compound, bottom_cp_compound_dose)

amlodipine 0.37 uM


In [7]:
# Least reproducible compound in L1000 (with at least 3 replicates)
bottom_l1000_compound = l1000_platemap_df.query("no_of_compounds >= 3").sort_values(by="median_score").compound.unique()[0]
bottom_l1000_compound_dose = l1000_platemap_df.query("no_of_compounds >= 3").sort_values(by="median_score").reset_index().dose[0]
print(bottom_l1000_compound, bottom_l1000_compound_dose)

fluphenazine 0.04 uM


In [8]:
# Random sample of DMSO
plate_metadata_dir = pathlib.Path("..", "..", "1.Data-exploration", "Profiles_level4", "plate_position_effects", "data")
cp_plate_metadata_file = pathlib.Path(plate_metadata_dir, "CellPainting_platemap_metadata.tsv.gz")

cp_plate_metadata_df = (
    pd.read_csv(cp_plate_metadata_file, sep="\t")
    .query("Metadata_broad_sample == 'DMSO'")
    .sample(n=5)
    .reset_index(drop=True)
    .rename(columns={
        "Metadata_Well": "well",
        "Metadata_broad_sample": "compound",
        "Metadata_dose_recode": "dose_recode"
    })
)

print(cp_plate_metadata_df.shape)
cp_plate_metadata_df.head()

(5, 8)


,Metadata_Assay_Plate_Barcode,well,Metadata_Plate_Map_Name,replicate_name,dose_recode,compound,pert_iname,moa
0,SQ00015205,B04,C-7161-01-LM6-021,replicate_1641,0,DMSO,dmso,control vehicle
1,SQ00015041,F09,C-7161-01-LM6-024,replicate_134,0,DMSO,dmso,control vehicle
2,SQ00014819,A04,C-7161-01-LM6-023,replicate_411,0,DMSO,dmso,control vehicle
3,SQ00015219,B06,C-7161-01-LM6-011,replicate_203,0,DMSO,dmso,control vehicle
4,SQ00015156,A02,C-7161-01-LM6-003,replicate_889,0,DMSO,dmso,control vehicle


In [9]:
# Compile target compound metadata details for Cell Painting
select_columns = [
    "compound", "dose_recode", "dose", "median_score", "moa", "well", "Metadata_Assay_Plate_Barcode"
]

cp_target_df = (
    pd.concat([
        cp_platemap_df.query("compound == @top_cp_compound").query("dose == @top_cp_compound_dose"),
        cp_platemap_df.query("compound == @bottom_cp_compound").query("dose == @bottom_cp_compound_dose"),
        cp_platemap_df.query("compound == @top_l1000_compound").query("dose == @top_l1000_compound_dose"),
        cp_platemap_df.query("compound == @bottom_l1000_compound").query("dose == @bottom_l1000_compound_dose"),
        cp_plate_metadata_df
    ], axis="rows")
    .reset_index(drop=True)
    .loc[:, select_columns]
)

cp_target_df

,compound,dose_recode,dose,median_score,moa,well,Metadata_Assay_Plate_Barcode
0,kpt-330,5,3.33 uM,0.923480,exportin antagonist,E14,SQ00015157
1,kpt-330,5,3.33 uM,0.923480,exportin antagonist,E14,SQ00015159
2,kpt-330,5,3.33 uM,0.923480,exportin antagonist,E14,SQ00015158
3,kpt-330,5,3.33 uM,0.923480,exportin antagonist,E14,SQ00015160
4,kpt-330,5,3.33 uM,0.923480,exportin antagonist,E14,SQ00015128
5,amlodipine,3,0.37 uM,-0.034522,calcium channel blocker,H10,SQ00015157
6,amlodipine,3,0.37 uM,-0.034522,calcium channel blocker,H10,SQ00015159
7,amlodipine,3,0.37 uM,-0.034522,calcium channel blocker,H10,SQ00015158
8,amlodipine,3,0.37 uM,-0.034522,calcium channel blocker,H10,SQ00015160
9,amlodipine,3,0.37 uM,-0.034522,calcium channel blocker,H10,SQ00015128


In [10]:
image_df = []
for idx, compound_row in cp_target_df.iterrows():
    # Metadata details
    cpd = compound_row.compound
    dose = compound_row.dose_recode
    score = compound_row.median_score
    moa = compound_row.moa
    
    # Define compound identifier (unique only internally to table)
    compound_id = f"ID{idx}"

    # Details required for XML parsing
    well = compound_row.well
    plate = compound_row.Metadata_Assay_Plate_Barcode
    
    # Identify the folder the xml plate info resides
    xml_folder = pathlib.Path("xml")
    xml_plate_folder = [x for x in xml_folder.iterdir() if plate in x.name][0]

    # Set the index file
    index_file = pathlib.Path(xml_plate_folder, "Images", "Index.idx.xml")
    
    # Setup the url extraction from the PerkinElmer class
    plate_xml = PerkinElmerXML(index_file)
    well_image_ids = plate_xml.get_well_image_ids(well)
    image_urls = plate_xml.extract_image_urls(well_image_ids)
    
    # Clean the result for downstream processing
    image_url_df = pd.DataFrame(image_urls, index=[0, 1]).transpose().reset_index()
    image_url_df.columns = ["image_id", "image_url", "image_channel"]

    image_url_df = image_url_df.assign(
        compound=cpd,
        compound_id=compound_id,
        dose=dose,
        median_score=score,
        moa=moa,
        well=well,
        plate=plate,
        xml_plate_folder_name=xml_plate_folder
    )
    
    # Obtain data for each well and plate
    image_df.append(image_url_df)

In [11]:
channels = list(get_channel_map().keys())
channels

['HOECHST 33342', 'Alexa 488', '488 long', 'Alexa 568', 'Alexa 647']

In [12]:
# Combine results and output
image_dfs = (
    pd.concat(image_df)
    .reset_index(drop=True)
)

# Select site 5 to only display 1 FOV
image_dfs = (
    image_dfs.assign(image_site=image_dfs.image_id.str[6:8])
    .query("image_site == 'F5'")
)

output_file = pathlib.Path("results", "representative_image_urls.tsv")
image_dfs.to_csv(output_file, sep="\t", index=False)

print(image_dfs.shape)
image_dfs.head(3)

(125, 12)


,image_id,image_url,image_channel,compound,compound_id,dose,median_score,moa,well,plate,xml_plate_folder_name,image_site
20,0514K1F5P1R1,r05c14f05p01-ch1sk1fk1fl1.tiff,HOECHST 33342,kpt-330,ID0,5,0.92348,exportin antagonist,E14,SQ00015157,xml/SQ00015157__2016-04-14T22_48_58-Measurement1,F5
21,0514K1F5P1R2,r05c14f05p01-ch2sk1fk1fl1.tiff,Alexa 488,kpt-330,ID0,5,0.92348,exportin antagonist,E14,SQ00015157,xml/SQ00015157__2016-04-14T22_48_58-Measurement1,F5
22,0514K1F5P1R3,r05c14f05p01-ch3sk1fk1fl1.tiff,488 long,kpt-330,ID0,5,0.92348,exportin antagonist,E14,SQ00015157,xml/SQ00015157__2016-04-14T22_48_58-Measurement1,F5
